In [2]:
import pandas as pd
from glob import glob

In [54]:
'''
1. Import raw data and concate them
2. There should be 101 stocks and 1720 days
3. There are some missing values
4. The values are represented in day percentage returns ((Close / Open) - 1) * 100
namely, how the stock price changes from the market open to market close each day
'''
intraday_return = pd.DataFrame()
prices = pd.DataFrame()
for file_name in glob(r'../../data/raw_market_data/*.csv'):
    df = pd.read_csv(file_name)
    ticker = file_name.split('/')[-1].split('.')[0]
    intraday_return[ticker] = ((df['close'] / df['open']) - 1) * 100
    prices[ticker] = df['adjclose']
intraday_return.index = df.iloc[:, 0]
intraday_return.index.name = "Date"
prices.index = df.iloc[:, 0]
prices.index.name = "Date"

print(intraday_return.shape)
intraday_return.head()


(1720, 101)


/var/folders/13/_z79rbzn6d1g6_qvmtzgrn3c0000gn/T/ipykernel_49961/4129245554.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  intraday_return[ticker] = ((df['close'] / df['open']) - 1) * 100
/var/folders/13/_z79rbzn6d1g6_qvmtzgrn3c0000gn/T/ipykernel_49961/4129245554.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  prices[ticker] = df['adjclose']


,CSCO,ISRG,PDD,LBTYK,VRTX,GILD,FOX,SWKS,MCHP,CDNS,...,TTWO,AAPL,DLTR,TMUS,SPLK,CTSH,MU,CDW,ASML,LRCX
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,-0.897344,-1.000226,0.754720,-0.820684,2.583978,-0.241745,1.554410,-0.380433,-1.080251,-1.671017,...,-0.354611,-1.849356,-0.831694,0.738003,-0.876750,-0.415960,-1.390470,-1.134424,-0.250763,-0.525862
2015-01-05,-0.951685,-1.608707,-10.675383,-4.690083,-1.014147,1.627471,1.164555,-1.326951,-1.768923,-1.656865,...,-3.662362,-1.883831,-0.826203,-0.700592,-2.470828,-3.020198,-2.679344,-0.662441,-1.918351,-0.996585
2015-01-06,-0.331614,-0.689722,-3.474901,-0.021682,-3.256118,-0.832741,1.159066,-3.282542,-1.691037,-3.031941,...,-0.781525,-0.262811,-1.590715,-0.591934,-2.380120,-2.168492,-3.607037,-1.449695,-1.841925,-1.769348
2015-01-07,0.589535,-0.217649,1.756754,-0.777539,1.548564,0.171187,-2.084829,-0.685599,-0.414365,2.106425,...,-0.142143,0.513063,2.385085,3.669725,-1.488614,2.811956,1.807795,1.195683,0.806128,0.362270
2015-01-08,-0.108935,0.832386,4.798763,-0.602286,1.614755,1.488095,-4.468604,3.055551,2.573527,0.000000,...,0.529847,2.435225,0.070367,2.471754,2.778269,1.968208,3.409081,1.425720,1.442026,1.660709


In [55]:
'''
1. Keep the stocks that have at least 80% non-Na values, drop the rest
2. Fill intraday Na values with zero, fill prices Na values using forward fill
3. There should be 95 stocks left
'''

DROP_THRESHOLD_PERCENTAGE = 0.8
drop_threshold = intraday_return.shape[0] * DROP_THRESHOLD_PERCENTAGE

intraday_return.dropna(axis=1, thresh=drop_threshold, inplace=True)
intraday_return.fillna(0.0, inplace=True)

prices.dropna(axis=1, thresh=drop_threshold, inplace=True)
prices.fillna(method='ffill', inplace=True)

print(intraday_return.shape)
print(prices.shape)


(1720, 95)
(1720, 95)


In [56]:
intraday_return.to_csv(
    r"../../data/clean_market_data/intraday_return.csv")
prices.to_csv(
    r"../../data/clean_market_data/prices.csv")


In [57]:
'''
1. Transpose the table so that:
    * Intraday_return become the features of each stock
    * Each stocks as an observation
'''

intraday_return_trans = intraday_return.transpose()
intraday_return_trans.index.name = "Tick"


In [58]:
intraday_return_trans.to_csv(r"../../data/clean_market_data/features.csv")